In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Distilledモデルとトークナイザーをロード
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

device="cuda"
model.to(device)

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.generation_config.pad_token_id = tokenizer.pad_token_id

/root/miniconda3/envs/my-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate_response(input_text, model, tokenizer, max_length=100, temperature=1.0):
    
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=100).to("cuda")  # 必要に応じてCUDAに送る
    output = model.generate(inputs["input_ids"], attention_mask=inputs['attention_mask'], max_length=max_length, num_return_sequences=1, temperature=temperature)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return response

In [3]:
inputs = tokenizer(["this is a"], return_tensors="pt", truncation=True, padding=True, max_length=20).to("cuda") 
print(inputs)
attention_mask=inputs['attention_mask']
print(attention_mask)
output = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=40, num_return_sequences=1, temperature=1.0 )
print(output)
response = tokenizer.decode(output[0])
print(response)

{'input_ids': tensor([[5661,  318,  257]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}
tensor([[1, 1, 1]], device='cuda:0')
tensor([[5661,  318,  257,  845,  922, 2126,  284,  779,  257, 1180, 3124, 7791,
          329,  262,  976, 1517,   13,  198,  198,  464,  717, 1517,  345,  761,
          284,  466,  318,  284,  751,  257,  649, 1627,  284,  262,  886,  286,
          262, 1627,   13,  198]], device='cuda:0')
this is a very good idea to use a different color scheme for the same thing.

The first thing you need to do is to add a new line to the end of the line.



In [5]:
user_input = ["明日の天気は"]
generate_response(user_input, model, tokenizer)

'明日の天気は、私は、私は、私は、私は、私は、私は、私は、私は、私は、私は、私は、私は、私は、私は、私は、私は、私は、私は、'

In [7]:
sequences = ['今日は天気が良いので外に出かけたいな。', '今日は暑い。']

sequence1_ids = tokenizer(sequences[0], return_tensors='pt').input_ids
sequence2_ids = tokenizer(sequences[1], return_tensors='pt').input_ids

print("1文目のtokenize結果: ", sequence1_ids) # 1文目のtokenize結果
print("2文目のtokenize結果: ", sequence2_ids) # ２文目のtokenize結果

# 入力をpaddingしてバッチにする
batched_ids = tokenizer(sequences, padding=True, return_tensors='pt').input_ids # padding=Trueで自動的にPaddingしてくれる

print("入力バッチのtokenize結果", batched_ids)

1文目のtokenize結果:  tensor([[20015,   232, 33768,    98, 31676, 25465, 36365,   245, 35585,   164,
           231,   107, 18566,  5641, 30640, 13783,   244, 28618, 49035,   118,
         27370,  2515,   239, 25224, 18566, 26945, 16764]])
2文目のtokenize結果:  tensor([[20015,   232, 33768,    98, 31676,   162,   248,   239, 18566, 16764]])
入力バッチのtokenize結果 tensor([[20015,   232, 33768,    98, 31676, 25465, 36365,   245, 35585,   164,
           231,   107, 18566,  5641, 30640, 13783,   244, 28618, 49035,   118,
         27370,  2515,   239, 25224, 18566, 26945, 16764],
        [20015,   232, 33768,    98, 31676,   162,   248,   239, 18566, 16764,
         50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257, 50257, 50257, 50257]])


In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM

device='cuda'
# load tokenizer
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")

# load pre-trained model
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

model.to(device)
# Set input word
input = tokenizer.encode("近年の人工知能が著しく上昇しているのはなぜ？", return_tensors="pt").to(device)

# inference
output = model.generate(input, do_sample=True, max_length=100)

# inferred output
print(tokenizer.batch_decode(output))

OSError: rinna/japanese-gpt2-large is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [26]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# モデルとトークナイザーの読み込み
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# 質問と文脈の設定
question = "What is color of apple?"

# トークン化
inputs = tokenizer.encode_plus(question, return_tensors="pt")

# 推論
with torch.no_grad():
    outputs = model(**inputs)

# 最も高いスコアのスパンを取得
start_scores = outputs.start_logits
end_scores = outputs.end_logits
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores) + 1  # end indexはinclusiveなので+1

# 答えをデコード
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_index:end_index]))
print(answer)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


color of apple ? [SEP]


In [28]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# モデルとトークナイザーの読み込み
model_name = "gpt2"  # または "gpt2-medium", "gpt2-large" など
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 質問の入力
question = "who are you？"
input_text = f"Q: {question} A:"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 推論の実行
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# 出力のデコード
answer = tokenizer.decode(output[0], skip_special_tokens=True)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Q: who are you？ A: I am a member of the United States Army. I am a member of the United States Army. I am a member of the United States Army. I am a member of the United States Army.
